<a href="https://colab.research.google.com/github/anapriba/MLLM-bilingual-explorations/blob/main/Zavrsni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

     |████████████████████████████████| 2.5MB 7.8MB/s 
     |████████████████████████████████| 3.3MB 47.8MB/s 
     |████████████████████████████████| 901kB 49.4MB/s 


In [3]:
pip install sentencepiece

In [2]:
import datetime
import json
import logging
import multiprocessing

import sys
from collections import defaultdict
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import torch

from pandas import DataFrame

from sklearn.manifold import TSNE
from torch import nn
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from tqdm import tqdm
from transformers import AutoConfig, AutoModel, AutoTokenizer
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os 
import csv
import pandas as pd
df = pd.read_csv("/content/Reviews_parallel2.tsv", sep="\t", )
#df.columns = ["text"]


Ana_sentences =[]
Ana_sentiment =[]

for sentence in df['text'].tolist():
  #print(sentence)
  Ana_sentences.append(sentence)

for sentence in df['translated text'].tolist():
  Ana_sentences.append(sentence)

print(Ana_sentences)

for sentence in df['sentiment'].tolist():
  #print(sentence)
  Ana_sentiment.append(sentence) # sentiment for english

for sentence in df['sentiment'].tolist():
  Ana_sentiment.append(sentence) # sentiment for croatian

print(Ana_sentiment)


['Read the book, forget the movie!', 'I hope this group of film-makers never re-unites.', 'Primary plot!Primary direction!Poor interpretation.', 'Brilliant and moving performances by Tom Courtenay and Peter Finch.', 'What a script, what a story, what a mess!', 'More suspenseful, more subtle, much, much more disturbing....', 'Just love the interplay between two great characters of stage & screen - Veidt & Barrymore', 'This is a great movie.', 'Too bad it is not available on home video.', 'a mesmerizing film that certainly keeps your attention... Ben Daniels is fascinating (and courageous) to watch.', 'One of the funniest movies made in recent years.', 'Good characterization, plot and exceptional chemistry make this one a classic', 'This is a good film.', 'This is very funny.', 'Yet after this film there were no good Ernest films!', 'For pure gothic vampire cheese nothing can compare to the Subspecies films.', 'I highly recommend each and every one of them.', 'Absolutely fantastic!', "Wh

In [ ]:
def run_transformer(model_name = "classla/bcms-bertic"):
    
    # Load AutoModel from huggingface model repository
    
    # model_name = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_config = AutoConfig.from_pretrained(
        model_name, output_hidden_states=True)
    model = AutoModel.from_pretrained(model_name, config=model_config)
    model.to("cuda")
    extracted_features = []
    training_generator = torch.utils.data.DataLoader(Ana_sentences, batch_size=512, num_workers=4) 
    for batch in training_generator: 
      print(batch)  

      # Tokenize sentences
      encoded_input = tokenizer(batch, padding="max_length", truncation=True, max_length=128, return_tensors='pt')
      encoded_input.to("cuda")
      with torch.no_grad():
        model_output = model(**encoded_input)
        extracted_features.extend(model_output.pooler_output.detach().cpu().numpy().tolist())
    # Compute token embeddings
    output_folder_name = model_name.split("/")[-1]
    with open(f'/content/All_sentences-{output_folder_name}.tsv', 'w', newline='') as f_output:
      tsv_output = csv.writer(f_output, delimiter='\t')
    
      for vector in extracted_features:
        tsv_output.writerow(vector)


In [4]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [8]:
def bertic_transformer(model_name = "classla/bcms-bertic"):
    
    # Load AutoModel from huggingface model repository
    
    # model_name = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_config = AutoConfig.from_pretrained(
        model_name, output_hidden_states=True)
    model = AutoModel.from_pretrained(model_name, config=model_config)
    model.to("cuda")
    extracted_features = []
    training_generator = torch.utils.data.DataLoader(Ana_sentences, batch_size=512, num_workers=4) 
    for batch in training_generator: 
      print(batch)  

      # Tokenize sentences
      encoded_input = tokenizer(batch, padding="max_length", truncation=True, max_length=128, return_tensors='pt')
      encoded_input.to("cuda")
      with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        extracted_features.extend(sentence_embeddings.detach().cpu().numpy().tolist())
    # Compute token embeddings
    output_folder_name = model_name.split("/")[-1]
    with open(f'/content/All_sentences-{output_folder_name}.tsv', 'w', newline='') as f_output:
      tsv_output = csv.writer(f_output, delimiter='\t')
    
      for vector in extracted_features:
        tsv_output.writerow(vector)


bertic_transformer()

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader i

['Read the book, forget the movie!', 'I hope this group of film-makers never re-unites.', 'Primary plot!Primary direction!Poor interpretation.', 'Brilliant and moving performances by Tom Courtenay and Peter Finch.', 'What a script, what a story, what a mess!', 'More suspenseful, more subtle, much, much more disturbing....', 'Just love the interplay between two great characters of stage & screen - Veidt & Barrymore', 'This is a great movie.', 'Too bad it is not available on home video.', 'a mesmerizing film that certainly keeps your attention... Ben Daniels is fascinating (and courageous) to watch.', 'One of the funniest movies made in recent years.', 'Good characterization, plot and exceptional chemistry make this one a classic', 'This is a good film.', 'This is very funny.', 'Yet after this film there were no good Ernest films!', 'For pure gothic vampire cheese nothing can compare to the Subspecies films.', 'I highly recommend each and every one of them.', 'Absolutely fantastic!', "Wh

In [5]:
def roberta_transformer(model_name = "sismetanin/xlm_roberta_base-ru-sentiment-rusentiment"):
    
    # Load AutoModel from huggingface model repository
    
    # model_name = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_config = AutoConfig.from_pretrained(
        model_name, output_hidden_states=True)
    model = AutoModel.from_pretrained(model_name, config=model_config)
    model.to("cuda")
    extracted_features = []
    training_generator = torch.utils.data.DataLoader(Ana_sentences, batch_size=128, num_workers=4) 
    for batch in training_generator: 
      print(batch) 

      # Tokenize sentences
      encoded_input = tokenizer(batch, padding="max_length", truncation=True, max_length=128, return_tensors='pt')
      encoded_input.to("cuda")
      with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        extracted_features.extend(sentence_embeddings.detach().cpu().numpy().tolist())
    # Compute token embeddings
    output_folder_name = model_name.split("/")[-1]
    with open(f'/content/All_sentences-{output_folder_name}.tsv', 'w', newline='') as f_output:
      tsv_output = csv.writer(f_output, delimiter='\t')
    
      for vector in extracted_features:
        tsv_output.writerow(vector)


roberta_transformer()

Some weights of the model checkpoint at sismetanin/xlm_roberta_base-ru-sentiment-rusentiment were not used when initializing XLMRobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at sismetanin/xlm_roberta_base-ru-sentiment-rusentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task 

['Read the book, forget the movie!', 'I hope this group of film-makers never re-unites.', 'Primary plot!Primary direction!Poor interpretation.', 'Brilliant and moving performances by Tom Courtenay and Peter Finch.', 'What a script, what a story, what a mess!', 'More suspenseful, more subtle, much, much more disturbing....', 'Just love the interplay between two great characters of stage & screen - Veidt & Barrymore', 'This is a great movie.', 'Too bad it is not available on home video.', 'a mesmerizing film that certainly keeps your attention... Ben Daniels is fascinating (and courageous) to watch.', 'One of the funniest movies made in recent years.', 'Good characterization, plot and exceptional chemistry make this one a classic', 'This is a good film.', 'This is very funny.', 'Yet after this film there were no good Ernest films!', 'For pure gothic vampire cheese nothing can compare to the Subspecies films.', 'I highly recommend each and every one of them.', 'Absolutely fantastic!', "Wh

In [ ]:
run_transformer()


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader i

['Read the book, forget the movie!', 'I hope this group of film-makers never re-unites.', 'Primary plot!Primary direction!Poor interpretation.', 'Brilliant and moving performances by Tom Courtenay and Peter Finch.', 'What a script, what a story, what a mess!', 'More suspenseful, more subtle, much, much more disturbing....', 'Just love the interplay between two great characters of stage & screen - Veidt & Barrymore', 'This is a great movie.', 'Too bad it is not available on home video.', 'a mesmerizing film that certainly keeps your attention... Ben Daniels is fascinating (and courageous) to watch.', 'One of the funniest movies made in recent years.', 'Good characterization, plot and exceptional chemistry make this one a classic', 'This is a good film.', 'This is very funny.', 'Yet after this film there were no good Ernest films!', 'For pure gothic vampire cheese nothing can compare to the Subspecies films.', 'I highly recommend each and every one of them.', 'Absolutely fantastic!', "Wh

AttributeError: ignored

In [ ]:
import os 
import csv
import pandas as pd
df = pd.read_csv("/content/Reviews_parallel.tsv", sep="\t")
#df.columns = ["text"]


Ana_sentences =[]
Ana_sentiment =[]

max_eng = len(df['text'])
eng_sent = df['text'].tolist()
for i in range(0, int(max_eng/2)):
    Ana_sentences.append(eng_sent[i])

max_hr = len(df['translated text'])
hr_sent = df['translated text'].tolist()
for i in range(0, int(max_hr/2)):
    Ana_sentences.append(hr_sent[i])


max_sent = len(df['sentiment'])
sent_sent = df['sentiment'].tolist()
for i in range(0, int(max_sent/2)):
    Ana_sentiment.append(sent_sent[i])

for i in range(0, int(max_sent/2)):
    Ana_sentiment.append(sent_sent[i])

print(Ana_sentences)
print(Ana_sentiment)


['Read the book, forget the movie!', 'I hope this group of film-makers never re-unites.', 'Primary plot!Primary direction!Poor interpretation.', 'Brilliant and moving performances by Tom Courtenay and Peter Finch.', 'What a script, what a story, what a mess!', 'More suspenseful, more subtle, much, much more disturbing....', 'Just love the interplay between two great characters of stage & screen - Veidt & Barrymore', 'This is a great movie.', 'Too bad it is not available on home video.', 'a mesmerizing film that certainly keeps your attention... Ben Daniels is fascinating (and courageous) to watch.', 'One of the funniest movies made in recent years.', 'Good characterization, plot and exceptional chemistry make this one a classic', 'This is a good film.', 'This is very funny.', 'Yet after this film there were no good Ernest films!', 'For pure gothic vampire cheese nothing can compare to the Subspecies films.', 'I highly recommend each and every one of them.', 'Absolutely fantastic!', "Wh

In [ ]:
textfile = open("/content/all_sentiments3.tsv", "w")
for element in Ana_sentiment:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
textfile = open("/content/ALL_sentences3.tsv", "w")
for element in Ana_sentences:
  textfile.write(element + "\n")
textfile.close()

In [ ]:
textfile = open("/content/ALL_sentences2.tsv", "w")
beksles = '\n'
for i, element in enumerate(Ana_sentences):
  if beksles in element:
    print(i)
    continue
  textfile.write(element + "\n")
textfile.close()

870


In [ ]:
textfile = open("/content/all_sentiments2.tsv", "w")
for i, element in enumerate(Ana_sentiment):
  if i == 870:
    continue
  textfile.write(element + "\n")
textfile.close()

In [ ]:
textfile = open("/content/all_meta.tsv", "w")
max = len(Ana_sentences)
textfile.write('sentence' + '\t' + 'sentiment' + '\t' + 'language' + '\n')
for i, meta in enumerate(Ana_sentences):
  language = 'English' if (i < max/2) else 'Croatian' 
  textfile.write(meta + '\t' + Ana_sentiment[i] + '\t' + language + '\n')
textfile.close()
print(max)

4086


In [ ]:
import os 
import csv
import pandas as pd
df = pd.read_csv("/content/Reviews_parallel.tsv", sep='\t')

In [ ]:
!wc -l All_sentences-bert-base-multilingual-uncased.tsv

4086 All_sentences-bert-base-multilingual-uncased.tsv


In [ ]:
df.head()
print(len(Ana_sentences))
print(len(Ana_sentiment))

4086
4086


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("sismetanin/xlm_roberta_base-ru-sentiment-rusentiment")

model = AutoModelForSequenceClassification.from_pretrained("sismetanin/xlm_roberta_base-ru-sentiment-rusentiment")

In [ ]:
model.roberta


In [14]:
model.roberta(**tokenizer(["this is test"], padding="max_length", truncation=True, max_length=128, return_tensors='pt'))

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[2.9659e-01, 8.8423e-01, 5.0535e-03,  ..., 1.2905e-01,
                                                         5.0509e-03, 8.9901e-01],
                                                        [3.7776e-01, 5.4042e-01, 4.3818e-01,  ..., 1.7503e+00,
                                                         3.0878e-01, 6.9863e-01],
                                                        [1.7411e-01, 4.7183e-01, 4.6771e-01,  ..., 1.9836e+00,
                                                         3.8516e-01, 8.5330e-01],
                                                        ...,
                                                        [2.9508e-01, 8.8136e-01, 2.1193e-03,  ..., 1.2275e-01,
                                                         3.1456e-05, 9.0306e-01],
                                                        [2.9508e-01, 8.8136e-01, 2.1193e-03,  ..., 